In [1]:
from chi import server, context, lease
import os, time

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [2]:
l = lease.get_lease(f"mlops-alstrain-project23")
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>mlops-alstrain…

Lease Details:
Name: mlops-alstrain-project23
ID: 95efb30c-c934-4c27-bfed-10e16a99d9f4
Status: ACTIVE
Start Date: 2025-05-10 13:41:00
End Date: 2025-05-11 04:00:00
User ID: f9a642531ef78b49697f8beaba43bd4081bc2584478a0f280c0577547b859bc6
Project ID: d3c6e101843a4ba79e665ebf59b521a2

Node Reservations:
ID: 035c74e6-025d-4ddc-97f8-25f4f1284e7d, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [4]:
username = "project23"
s = server.Server(
    f"node-alstrain-{username}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-hwe",
    key_name="id_rsa_project23"
)
s.submit(idempotent=True)

Waiting for server node-alstrain-project23's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-alstrain-project23
Id,964fa117-8ba0-4f05-be15-b2c5275a7bd1
Status,ACTIVE
Image Name,CC-Ubuntu24.04-hwe
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.251 (v4) Type: fixed MAC: bc:97:e1:c4:c3:40
Network Name,sharednet1
Created At,2025-05-10T14:11:49Z
Keypair,id_rsa_project23
Reservation Id,None
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


In [5]:
s.associate_floating_ip()

In [6]:
s.refresh()
s.check_connectivity()
# ssh -i ~/.ssh/id_rsa_project23 cc@129.114.109.50

Checking connectivity to 129.114.109.50 port 22.


Connection successful


In [7]:
s.refresh()
s.show(type="widget")

Attribute,node-alstrain-project23
Id,964fa117-8ba0-4f05-be15-b2c5275a7bd1
Status,ACTIVE
Image Name,CC-Ubuntu24.04-hwe
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.251 (v4) Type: fixed MAC: bc:97:e1:c4:c3:40 IP: 129.114.109.50 (v4) Type: floating MAC: bc:97:e1:c4:c3:40
Network Name,sharednet1
Created At,2025-05-10T14:11:49Z
Keypair,id_rsa_project23
Reservation Id,035c74e6-025d-4ddc-97f8-25f4f1284e7d
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


## Setting up Docker

In [8]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.109.50: b'e2bf6ead33a4a2848152a75351b6a9cc'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

## Pull and start Jupyter/PySpark container

In [9]:
s.execute("docker pull jupyter/pyspark-notebook:latest")

latest: Pulling from jupyter/pyspark-notebook
aece8493d397: Pulling fs layer
fd92c719666c: Pulling fs layer
088f11eb1e74: Pulling fs layer
4f4fb700ef54: Pulling fs layer
ef8373d600b0: Pulling fs layer
77e45ee945dc: Pulling fs layer
a30f89a0af6c: Pulling fs layer
dc42adc7eb73: Pulling fs layer
abaa8376a650: Pulling fs layer
aa099bb9e49a: Pulling fs layer
822c4cbcf6a6: Pulling fs layer
d25166dcdc7b: Pulling fs layer
ef8373d600b0: Waiting
964fc3e4ff9f: Pulling fs layer
2c4c69587ee4: Pulling fs layer
de2cdd875fa8: Pulling fs layer
75d33599f5f2: Pulling fs layer
77e45ee945dc: Waiting
a30f89a0af6c: Waiting
dc42adc7eb73: Waiting
31973ea82470: Pulling fs layer
4f4fb700ef54: Waiting
96ee7e4439c7: Pulling fs layer
1f9ad23c07ac: Pulling fs layer
75d33599f5f2: Waiting
d25166dcdc7b: Waiting
aa099bb9e49a: Waiting
31973ea82470: Waiting
d19266e0cb17: Pulling fs layer
96ee7e4439c7: Waiting
9a165b6e9dc7: Pulling fs layer
5689442fd4e1: Pulling fs layer
9a6a202f62a6: Pulling fs layer
1f9ad23c07ac: Waiting

<Result cmd='docker pull jupyter/pyspark-notebook:latest' exited=0>

    docker run -it --rm -p 8888:8888 \
    --name pysparknb \
    -v ~/my_workspace:/home/jovyan/work \
    -v /mnt/object:/home/jovyan/work/data \
    -u 1000:1000 \
    jupyter/pyspark-notebook